In [2]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Load Datasets

## Load Train Data

In [3]:
df_train = pd.read_csv('Datasets/train.csv')
df_train

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1230,yes,adult,535246,38.5,129.0,48.0,cool,reduced,pale_pink,...,57.0,66.0,serosanguious,2.0,yes,2206,0,0,no,lived
1231,1231,yes,adult,528570,37.5,60.0,50.0,cool,reduced,pale_cyanotic,...,35.0,6.4,serosanguious,3.6,yes,2209,0,0,yes,died
1232,1232,yes,young,529685,37.5,84.0,40.0,normal,reduced,normal_pink,...,40.0,5.9,cloudy,7.0,yes,400,0,0,yes,lived
1233,1233,yes,adult,534784,38.1,70.0,16.0,normal,reduced,bright_red,...,58.0,74.0,cloudy,2.0,yes,2209,0,0,no,lived


In [4]:
df_X_train = df_train.iloc[:,:-1]
df_y_train = df_train.iloc[:,-1]
df_X_train.drop('id',inplace=True,axis=1)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1196 non-null   object 
 8   peripheral_pulse       1175 non-null   object 
 9   mucous_membrane        1214 non-null   object 
 10  capillary_refill_time  1229 non-null   object 
 11  pain                   1191 non-null   object 
 12  peristalsis            1215 non-null   object 
 13  abdominal_distention   1212 non-null   object 
 14  nasogastric_tube       1155 non-null   object 
 15  naso

In [6]:
del df_train

## Load Test Data

In [7]:
df_test = pd.read_csv('Datasets/test.csv')
df_test

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,2054,no,adult,529461,40.3,114.0,36.0,cool,reduced,normal_pink,...,distend_large,57.0,8.1,serosanguious,4.5,yes,3205,0,0,yes
820,2055,yes,adult,535338,37.2,100.0,20.0,cool,reduced,pale_cyanotic,...,distend_small,50.0,66.0,serosanguious,2.0,yes,2209,0,0,no
821,2056,yes,adult,529640,39.2,132.0,12.0,cool,reduced,dark_cyanotic,...,NaN,53.0,7.6,serosanguious,4.5,yes,2205,0,0,no
822,2057,no,adult,5287179,38.3,54.0,66.0,normal,normal,normal_pink,...,NaN,49.0,8.6,clear,5.0,no,3111,0,0,yes


In [8]:
df_test_id = df_test.pop('id')

In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                824 non-null    object 
 1   age                    824 non-null    object 
 2   hospital_number        824 non-null    int64  
 3   rectal_temp            824 non-null    float64
 4   pulse                  824 non-null    float64
 5   respiratory_rate       824 non-null    float64
 6   temp_of_extremities    789 non-null    object 
 7   peripheral_pulse       777 non-null    object 
 8   mucous_membrane        811 non-null    object 
 9   capillary_refill_time  818 non-null    object 
 10  pain                   795 non-null    object 
 11  peristalsis            805 non-null    object 
 12  abdominal_distention   802 non-null    object 
 13  nasogastric_tube       760 non-null    object 
 14  nasogastric_reflux     810 non-null    object 
 15  nasoga

# Data Preprocessing

- ## Handle Missing Value

In [10]:
def handle_missing_value(df: pd.DataFrame):
    for i in df.columns:
        if pd.api.types.is_numeric_dtype(df[i].dtype):
            df[i].fillna(df[i].mean(),inplace=True)
        else:
            df[i].fillna(df[i].mode()[0],inplace=True)

In [11]:
handle_missing_value(df_X_train)
df_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                1235 non-null   object 
 1   age                    1235 non-null   object 
 2   hospital_number        1235 non-null   int64  
 3   rectal_temp            1235 non-null   float64
 4   pulse                  1235 non-null   float64
 5   respiratory_rate       1235 non-null   float64
 6   temp_of_extremities    1235 non-null   object 
 7   peripheral_pulse       1235 non-null   object 
 8   mucous_membrane        1235 non-null   object 
 9   capillary_refill_time  1235 non-null   object 
 10  pain                   1235 non-null   object 
 11  peristalsis            1235 non-null   object 
 12  abdominal_distention   1235 non-null   object 
 13  nasogastric_tube       1235 non-null   object 
 14  nasogastric_reflux     1235 non-null   object 
 15  naso

In [12]:
handle_missing_value(df_test)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                824 non-null    object 
 1   age                    824 non-null    object 
 2   hospital_number        824 non-null    int64  
 3   rectal_temp            824 non-null    float64
 4   pulse                  824 non-null    float64
 5   respiratory_rate       824 non-null    float64
 6   temp_of_extremities    824 non-null    object 
 7   peripheral_pulse       824 non-null    object 
 8   mucous_membrane        824 non-null    object 
 9   capillary_refill_time  824 non-null    object 
 10  pain                   824 non-null    object 
 11  peristalsis            824 non-null    object 
 12  abdominal_distention   824 non-null    object 
 13  nasogastric_tube       824 non-null    object 
 14  nasogastric_reflux     824 non-null    object 
 15  nasoga

- ## Label Encoding

In [33]:
# Encode Binary Data
df_X_train['surgery'].replace({'yes':1,'no':0},inplace=True)
df_X_train['age'].replace({'adult':1,'young':0},inplace=True)
df_X_train['surgical_lesion'].replace({'yes':1,'no':0},inplace=True)
df_X_train['cp_data'].replace({'yes':1,'no':0},inplace=True)
df_test['surgery'].replace({'yes':1,'no':0},inplace=True)
df_test['age'].replace({'adult':1,'young':0},inplace=True)
df_test['surgical_lesion'].replace({'yes':1,'no':0},inplace=True)
df_test['cp_data'].replace({'yes':1,'no':0},inplace=True)

# Encode Ordinal Data
df_X_train['temp_of_extremities'].replace({'cold':3,'cool':2,'normal':1,'warm':0},inplace=True)
df_X_train['peripheral_pulse'].replace({'reduced':3,'normal':2,'absent':1,'increased':0},inplace=True)
df_X_train['capillary_refill_time'].replace({'less_3_sec':1,'more_3_sec':2,'3':0},inplace=True)
df_X_train['pain'].replace({'extreme_pain': 4, 'severe_pain': 3, 'moderate': 2, 'depressed': 1, 'mild_pain': 1, 'slight': 0, 'alert': 0}, inplace=True)
df_X_train['peristalsis'].replace({'absent': 0, 'distend_small': 1, 'hypomotile': 2, 'normal': 3, 'hypermotile': 4}, inplace=True)
df_X_train['abdominal_distention'].replace({'none':0,'slight':1,'moderate':2,'severe':3},inplace=True)
df_X_train['nasogastric_tube'].replace({'none':0,'slight':1,'significant':2},inplace=True)
df_X_train['nasogastric_reflux'].replace({'none':0,'slight':1,'less_1_liter':2,'more_1_liter':3},inplace=True)
df_X_train['rectal_exam_feces'].replace({'absent':0,'decreased':1,'normal':2,'increased':3,'serosanguious':3},inplace=True)
df_X_train['abdomen'].replace({'normal':0,'firm':1,'distend_small':2,'distend_large':3,'other':0},inplace=True)
df_X_train['abdomo_appearance'].replace({'clear':0,'cloudy':1,'serosanguious':2},inplace=True)
df_X_train['mucous_membrane'].replace({'normal_pink':0,'bright_pink':1,'pale_pink':2,'pale_cyanotic':3,'bright_red':4,'dark_cyanotic':5},inplace=True)
df_test['temp_of_extremities'].replace({'cold':3,'cool':2,'normal':1,'warm':0},inplace=True)
df_test['peripheral_pulse'].replace({'reduced':3,'normal':2,'absent':1,'increased':0},inplace=True)
df_test['capillary_refill_time'].replace({'less_3_sec':1,'more_3_sec':2,'3':0},inplace=True)
df_test['pain'].replace({'extreme_pain': 4, 'severe_pain': 3, 'moderate': 2, 'depressed': 1, 'mild_pain': 1, 'slight': 0, 'alert': 0}, inplace=True)
df_test['peristalsis'].replace({'absent': 0, 'distend_small': 1, 'hypomotile': 2, 'normal': 3, 'hypermotile': 4}, inplace=True)
df_test['abdominal_distention'].replace({'none':0,'slight':1,'moderate':2,'severe':3},inplace=True)
df_test['nasogastric_tube'].replace({'none':0,'slight':1,'significant':2},inplace=True)
df_test['nasogastric_reflux'].replace({'none':0,'slight':1,'less_1_liter':2,'more_1_liter':3},inplace=True)
df_test['rectal_exam_feces'].replace({'absent':0,'decreased':1,'normal':2,'increased':3,'serosanguious':3},inplace=True)
df_test['abdomen'].replace({'normal':0,'firm':1,'distend_small':2,'distend_large':3,'other':0},inplace=True)
df_test['abdomo_appearance'].replace({'clear':0,'cloudy':1,'serosanguious':2},inplace=True)
df_test['mucous_membrane'].replace({'normal_pink':0,'bright_pink':1,'pale_pink':2,'pale_cyanotic':3,'bright_red':4,'dark_cyanotic':5},inplace=True)

df_X_train

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1,1,530001,38.1,132.0,24.0,2,3,5,2,...,2,57.0,8.5,2,3.4,1,2209,0,0,0
1,1,1,533836,37.5,88.0,12.0,2,2,3,2,...,2,33.0,64.0,2,2.0,1,2208,0,0,0
2,1,1,529812,38.3,120.0,28.0,2,3,2,1,...,3,37.0,6.4,2,3.4,1,5124,0,0,0
3,1,1,5262541,37.1,72.0,30.0,3,3,2,2,...,2,53.0,7.0,1,3.9,1,2208,0,0,1
4,0,1,5299629,38.0,52.0,48.0,1,2,0,1,...,0,47.0,7.3,1,2.6,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,1,1,535246,38.5,129.0,48.0,2,3,2,2,...,3,57.0,66.0,2,2.0,1,2206,0,0,0
1231,1,1,528570,37.5,60.0,50.0,2,3,3,1,...,2,35.0,6.4,2,3.6,1,2209,0,0,1
1232,1,0,529685,37.5,84.0,40.0,1,3,0,1,...,1,40.0,5.9,1,7.0,1,400,0,0,1
1233,1,1,534784,38.1,70.0,16.0,1,3,4,1,...,2,58.0,74.0,1,2.0,1,2209,0,0,0
